In [1]:
import pandas as pd
import numpy as np
import xarray as xr
## spatial filtering
import geopandas as gpd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('seaducks'), '..')))
from seaducks import add_grid_box_corners_to_df, assign_each_position_a_bin,get_corners, format_coordinates
from seaducks.data_processing.derived_quantities import sst_gradient_pointwise,interpolate_sst_gradient

In [2]:
# load drifter dataset
drifter_dataset = pd.read_hdf('../data/corrected_velocity_drifter_fifth_set_sst_gradient.h5')
# load sst data into a dictionary indexed by year
arrays_by_year = {}

file_directory = r"D:\PhD\ocean-datasets\copernicus-data"

filename="CMEMS_West_NA_sst_1993.nc"
output_path = os.path.join(file_directory, filename)
    
dataset = xr.open_dataset(output_path)
sst_array = dataset['analysed_sst']

# ensure correct look up (remove issues with machine precision)
lat_str = sst_array.coords['latitude'].values.astype(str)  # Convert lat values to strings
lon_str = sst_array.coords['longitude'].values.astype(str)  # Convert lat values to strings
# assign string coordinates
sst_array = sst_array.assign_coords(lat_str=('latitude', lat_str))
sst_array = sst_array.assign_coords(lon_str=('longitude', lon_str))


In [6]:
print(6*(2019-1993+1))

162


In [3]:
h = 0.05
lat_val_str = '0.725'
lon_val_str = '-82.875'
#lat_neighbours = [format_coordinates(float(lat_val_str)+jj*h) for jj in np.arange(-2,3,1)]
#lon_neighbours = [format_coordinates(float(lon_val_str)+jj*h) for jj in np.arange(-1,2,1)]
#sst_x_neighbours = [float(sst_array.isel(time=0).sel(latitude=lat_val_str, longitude=lon_val).values) if -83< float(lon_val)<-40 else np.nan for lon_val in lon_neighbours]
#sst_y_neighbours = [float(sst_array.isel(time=0).sel(latitude=lat_val, longitude=lon_val_str).values) if 0 < float(lat_val) < 60 else np.nan for lat_val in lat_neighbours]

In [4]:
from seaducks.data_processing.derived_quantities import sst_gradient_pointwise

coord_str = (f"{lat_val_str}",f"{lon_val_str}")
time_val = sst_array.time[0].values

sst_gradient_pointwise(sst_array,coord_str,time_val)

(-0.006295755114965829, 0.0008993215858197345)

In [4]:
# load drifter dataset
drifter_dataset = pd.read_hdf('../data/corrected_velocity_drifter_fifth_set_sst_gradient.h5')

In [17]:
np.mean(drifter_dataset.query('sst_y_derivative == sst_y_derivative and -75<lon<-70 and 30<lat<45 and 2<= time.dt.month <=4 ')['sst_y_derivative'])

-0.03368238451334288

In [3]:
# spatially restrict dataset
lon_lim_W = -83
lon_lim_E = -40
lat_lim_S = 0
lat_lim_N = 60

df = drifter_dataset.query('@lon_lim_W < lon < @lon_lim_E').copy()
df = df.query('@lat_lim_S < lat < @lat_lim_N').copy()


In [5]:
# latitude and longitude grids for sst array
lat_grid = np.array(sst_array['latitude'])
lon_grid = np.array(sst_array['longitude'])
bin_size = 0.05

In [6]:
drifter_df = assign_each_position_a_bin(df,lat_grid,lon_grid,bin_size = bin_size)

In [6]:
def test_get_corners(lat_cut: tuple, lon_cut: tuple) -> np.ndarray:
    '''
    Given a longitudinal and a latitudinal interval, returns the corners of the square
    spanned by the two intervals.

    Parameters
    ----------
    lat_cut: tuple
        Output of a pd.cut (latitude interval)
    lon_cut: tuple
        Output of a pd.cut (longitude interval)

    Returns
    -------
    np.ndarray
        Returns an array of coordinates of the corners of the square.
    '''
    
    if np.array([type(lat_cut) is not float,type(lon_cut) is not float]).all():
        lon1, lon2 = lon_cut.left, lon_cut.right
        lat1, lat2 = lat_cut.left, lat_cut.right
        return np.array([(lat1, lon1), (lat1, lon2), (lat2, lon2), (lat2, lon1)])
    else:
        nan_tuple = (np.nan,np.nan)
        return np.array([nan_tuple for ii in range(4)])

In [7]:
variables = list(drifter_df.columns)
grouped_df = drifter_df.apply(lambda x:get_corners(x[f"lat_bin_size_{bin_size}"],x[f"lon_bin_size_{bin_size}"]),axis=1)
# corners = [get_corners(multi_index) for multi_index in grouped_df.indices.keys()]

In [8]:
drifter_df.loc[:,'corners'] = grouped_df

In [10]:
new_grouped = df.apply(lambda row: interpolate_sst_gradient(format_coordinates(row['lat']),format_coordinates(row['lon']),row['time'],[sst_array],row['corners']),
    axis=1,
    result_type='expand')


KeyError: "not all values found in index 'time'. Try setting the `method` keyword argument (example: method='nearest')."